In [70]:
import numpy as np
import pandas as pd
import numpy.typing as npt
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
from definitions import ALPHA
from battery_data import test, train

In [71]:
DECIMAL_PLACES = 2

def index_to_predictor_variable(index: int) -> str:
    if index == 0:
        return 'intercept'
    
    return f'x{index}'

def mse(y: npt.NDArray, y_pred: npt.NDArray) -> float:
    n = len(y)
    return 1 / n * np.sum((y - y_pred) ** 2)

def smape(y: npt.NDArray, y_pred: npt.NDArray) -> float:
    n = len(y)
    return 1 / n * np.sum(np.abs(y - y_pred) / ((np.abs(y) + np.abs(y_pred)) / 2)) * 100

### Forward regression

In [72]:
MIN_POLYNOMIAL_DEGREE = 2
MAX_POLYNOMIAL_DEGREE = 10
DECIMAL_PLACES = 2

model_statistics = pd.DataFrame(columns=[
    'insignificant_predictors',
    'r_squared',
    'r_squared_adjusted',
    'aic',
    'bic',
    'f_statistic_p_value',
    'mse',
    'mse_test',
    'smape',
    'smape_test',
])
model_statistics.index.names = ['polynomial_degree']

for polynomial_degree in range(MIN_POLYNOMIAL_DEGREE, MAX_POLYNOMIAL_DEGREE + 1):
    polynomial_transformer = PolynomialFeatures(degree=polynomial_degree, include_bias=True) # Set include_bias=True to add a column of 1s to the array, such that the polynomial model becomes: Y = β_0 * 1 + β_1 * x^1 + ... + β_p * x^p + e. This column of 1s represents the intercept term β_0
    x = train.logged['capacity'].to_numpy()
    X = polynomial_transformer.fit_transform(x.reshape((-1, 1)))
    y = train.logged['RUL'].to_numpy()
    model = sm.OLS(y, X).fit()
    y_pred = model.predict(X)

    X_test = polynomial_transformer.fit_transform(test.logged['capacity'].to_numpy().reshape((-1, 1)))
    y_test = test.logged['RUL'].to_numpy()
    y_test_pred = model.predict(X_test)

    insignificant_predictors = [index_to_predictor_variable(index) for index, p_value in enumerate(model.pvalues) if p_value > ALPHA]
    insignificant_predictors_str = ','.join(insignificant_predictors) if insignificant_predictors else 'None'

    statistics_models_new = pd.DataFrame({
        'insignificant_predictors': insignificant_predictors_str,
        'r_squared': model.rsquared,
        'r_squared_adjusted': model.rsquared_adj,
        'aic': model.aic,
        'bic': model.bic,
        'f_statistic_p_value': model.f_pvalue,
        'mse': model.mse_resid,
        'mse_test': mse(y_test, y_test_pred),
        'smape': smape(y, y_pred),
        'smape_test': smape(y_test, y_test_pred),
    }, index=[polynomial_degree])
    statistics_models_new.index.names = ['polynomial_degree']
    model_statistics = pd.concat([model_statistics, statistics_models_new])

model_statistics_rounded = model_statistics.copy()
numeric_columns = [column for column in model_statistics_rounded if model_statistics_rounded[column].dtype == 'float64']
model_statistics_rounded[numeric_columns] = model_statistics_rounded[numeric_columns].apply(lambda x: np.round(x, DECIMAL_PLACES))
display(model_statistics_rounded)

,insignificant_predictors,r_squared,r_squared_adjusted,aic,bic,f_statistic_p_value,mse,mse_test,smape,smape_test
polynomial_degree,,,,,,,,,,
2,None,0.73,0.73,85363.26,85389.69,0.0,0.33,0.33,9.13,8.71
3,None,0.74,0.74,84664.21,84699.44,0.0,0.32,0.33,9.08,8.66
4,None,0.75,0.75,82798.30,82842.34,0.0,0.31,0.30,8.87,8.22
5,None,0.75,0.75,82369.84,82422.69,0.0,0.31,0.31,8.78,8.22
6,None,0.75,0.75,82362.83,82424.49,0.0,0.31,0.31,8.78,8.22
7,None,0.75,0.75,81997.70,82068.17,0.0,0.31,0.30,8.71,8.11
8,None,0.75,0.75,81942.58,82021.85,0.0,0.31,0.30,8.70,8.11
9,"intercept,x1,x2,x3,x4,x5,x6,x7,x8,x9",0.75,0.75,81999.20,82087.28,0.0,0.31,0.31,8.67,8.18
10,"intercept,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10",0.75,0.75,81945.54,82033.62,0.0,0.31,0.30,8.70,8.12


### Coefficient estimates

In [73]:
POLYNOMIAL_DEGREE = 8

polynomial_transformer = PolynomialFeatures(degree=POLYNOMIAL_DEGREE, include_bias=True)
x = train.logged['capacity'].to_numpy()
X = polynomial_transformer.fit_transform(x.reshape((-1, 1)))
y = train.logged['RUL'].to_numpy()
model = sm.OLS(y, X).fit()

scientific_round_vectorized = np.vectorize(np.format_float_scientific)
display(scientific_round_vectorized(model.params, DECIMAL_PLACES))
display(model.summary())

array(['-2.78e+08', '2.22e+09', '-7.77e+09', '1.55e+10', '-1.93e+10',
       '1.54e+10', '-7.66e+09', '2.18e+09', '-2.70e+08'], dtype='<U9')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                 1.866e+04
Date:                Sun, 16 Apr 2023   Prob (F-statistic):               0.00
Time:                        00:19:06   Log-Likelihood:                -40962.
No. Observations:               49433   AIC:                         8.194e+04
Df Residuals:                   49424   BIC:                         8.202e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.776e+08   3.21e+07     -8.636      0.000   -3.41e+08   -2.15e+08
x1          2.221e+09   2.61e+08      8.502      0.000    1.71e+09    2.73e+09
x2         -7.766e+09   9.28e+08     -8.367      0.000   -9.59e+09   -5.95e+09
x3           1.55e+10   1.88e+09      8.233      0.000    1.18e+10    1.92e+10
x4         -1.933e+10   2.39e+09     -8.100      0.000    -2.4e+10   -1.46e+10
x5           1.54e+10   1.93e+09      7.966      0.000    1.16e+10    1.92e+10
x6         -7.664e+09   9.78e+08     -7.833      0.000   -9.58e+09   -5.75e+09
x7          2.177e+09   2.83e+08      7.700      0.000    1.62e+09    2.73e+09
x8         -2.703e+08   3.57e+07     -7.567      0.000    -3.4e+08      -2e+08
==============================================================================
Omnibus:                     3641.356   Durbin-Watson:                   0.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1984.648
Skew:                           0.339   Prob(JB):                         0.00
Kurtosis:                       2.290   Cond. No.                     5.46e+12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.06e-20. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""